In [1]:
!pip install pyreadr

     |████████████████████████████████| 361 kB 4.4 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!cp /content/drive/My\ Drive/genomics_data/OCD_Dummy.rds content/genomics_data/OCD_Dummy.rds
!cp /content/drive/My\ Drive/genomics_data/Genetic_Dummy.rds content/genomics_data/Genetic_Dummy.rds
!cp /content/drive/My\ Drive/genomics_data/Batch_Dummy.rds content/genomics_data/Batch_Dummy.rds

In [17]:
import pandas as pd
import numpy as np
import pyreadr

In [18]:
def getData(data):
  return pyreadr.read_r(data)[None]

In [19]:
batch_df = getData("content/genomics_data/Batch_Dummy.rds")
genetic_df = getData("content/genomics_data/Genetic_Dummy.rds")
ocd_df = getData("content/genomics_data/OCD_Dummy.rds")

In [20]:
batch_df.head()

,SampleID,OCD,Experimental,Test,Valid,Train_1,Train_2,Train_3,Train_4,Train_5,...,Train_70,Train_71,Train_72,Train_73,Train_74,Train_75,Train_76,Train_77,ALL,TRAIN
0,ZLJF044767,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,NUZB250104,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,JOOL495893,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,ZKMU498502,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,XYJT184132,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [21]:
genetic_df.head()

,SampleID,rs0728654545,rs9192343147,rs9951538456,rs9692676705,rs6299711483,rs7224000387,rs2486054470,rs2441544862,rs1229871093,...,rs5491526932,rs7331227592,rs4500659624,rs7619812047,rs7224062992,rs5310501663,rs2000209651,rs3199655646,rs9411634933,rs5428412367
0,ZLJF044767,1.000,1.884,2.0,0.0,0.857,1.000,1.000,2.000,0.0,...,0.000,0.000,1.000,0.995,0.000,1.0,2.000,0.016,0.0,2.000
1,NUZB250104,0.957,2.000,1.0,2.0,2.000,1.000,0.998,1.000,0.0,...,1.926,0.999,1.208,1.127,0.000,0.0,0.000,0.000,2.0,0.000
2,JOOL495893,1.000,0.000,2.0,1.0,0.000,0.000,0.037,0.997,1.0,...,0.007,0.000,0.000,1.000,1.875,0.0,0.000,1.000,2.0,1.038
3,ZKMU498502,1.909,1.000,1.0,1.0,0.000,1.000,0.000,0.000,2.0,...,0.000,0.000,0.000,1.000,2.000,1.0,0.002,0.000,0.0,1.996
4,XYJT184132,0.000,1.000,2.0,0.0,0.014,0.421,2.000,0.000,1.0,...,0.000,0.978,0.000,1.000,0.889,0.0,1.069,1.000,0.0,0.577


In [22]:
ocd_df.head()

,SampleID,OCD
0,ZLJF044767,0
1,NUZB250104,0
2,JOOL495893,0
3,ZKMU498502,0
4,XYJT184132,0


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import accuracy_score
train_coef = []
train_intercept = []
for i in range(1,78):
  sample_ids = list(batch_df.loc[batch_df['Train_'+str(i)]==1]['SampleID'])
  genetic_np = genetic_df.loc[genetic_df['SampleID'].isin(sample_ids)].to_numpy()[:,1:]
  ocd_np = np.concatenate(ocd_df.loc[ocd_df['SampleID'].isin(sample_ids)].to_numpy()[:,1:])
  ocd_np[ocd_np==2]=1
  ocd_np = ocd_np.astype('int')
  model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.8).fit(genetic_np,ocd_np)
  train_coef.append(model.coef_)
  train_intercept.append(model.intercept_)
  print("Progress ===> ",i," model completed")
print(train_coef)
print(train_intercept)
train_coef = np.concatenate(train_coef)
train_intercept = np.concatenate(train_intercept)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


Progress ===>  1  model completed


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


Progress ===>  2  model completed


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


Progress ===>  75  model completed


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


Progress ===>  76  model completed
Progress ===>  77  model completed
[array([[-0.00456901,  0.02246858,  0.        , ...,  0.01107887,
         0.        ,  0.        ]]), array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.01035702]]), array([[ 0.        ,  0.00236031,  0.01732264, ...,  0.00609833,
         0.01713024, -0.01661748]]), array([[0.        , 0.00162235, 0.        , ..., 0.        , 0.        ,
        0.        ]]), array([[ 0.        ,  0.0127056 ,  0.        , ...,  0.        ,
        -0.01820864, -0.02502932]]), array([[ 0.        ,  0.        ,  0.00446013, ...,  0.        ,
         0.        , -0.03353528]]), array([[-0.00201339,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]]), array([[0.        , 0.00374664, 0.        , ..., 0.        , 0.        ,
        0.00107971]]), array([[0.        , 0.01250403, 0.        , ..., 0.        , 0.01328866,
        0.        ]]), array([[0.        , 0.0097255

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [24]:
from sklearn.metrics import confusion_matrix
train_model_coef = np.mean(train_coef, axis=0)
train_model_intercept = np.mean(train_intercept)

sample_ids = list(batch_df.loc[batch_df['Valid']==1]['SampleID'])
genetic_train_np = genetic_df.loc[genetic_df['SampleID'].isin(sample_ids)].to_numpy()[:,1:]
ocd_train_np = np.concatenate(ocd_df.loc[ocd_df['SampleID'].isin(sample_ids)].to_numpy()[:,1:])
ocd_train_np[ocd_train_np==2]=1
train_y_prob=[]
for i in range(len(genetic_train_np)):
  train_y_prob.append(1/(1+np.exp(-(np.dot(genetic_train_np[i],train_model_coef)+train_model_intercept))))
train_y_pred=np.array(train_y_prob)
train_y_pred[train_y_pred>0.5]=1
train_y_pred[train_y_pred<=0.5]=0
train_y_pred = train_y_pred.astype('int64')
ocd_train_np = ocd_train_np.astype('int64')
print(accuracy_score(ocd_train_np,train_y_pred))
print(confusion_matrix(ocd_train_np,train_y_pred))

0.61
[[23 27]
 [12 38]]


In [25]:
tn, fp, fn, tp = confusion_matrix(ocd_train_np,train_y_pred).ravel()
ck = (2*((tp * tn) - (fn * fp)))/(((tp + fp) * (fp + tn)) + ((tp + fn) * (fn+tn)))
print(ck)

0.19130434782608696


In [29]:
#Train Accuracy
genetic_train = []
ocd_train = []
for i in range(1,78):
  sample_ids = list(batch_df.loc[batch_df['Train_'+str(i)]==1]['SampleID'])
  genetic_train.append(genetic_df.loc[genetic_df['SampleID'].isin(sample_ids)].to_numpy()[:,1:])
  ocd_train.append(np.concatenate(ocd_df.loc[ocd_df['SampleID'].isin(sample_ids)].to_numpy()[:,1:]))
genetic_train = np.concatenate(genetic_train)
ocd_train = np.concatenate(ocd_train)
ocd_train[ocd_train==2]=1

y_prob=[]
for i in range(len(genetic_train)):
  y_prob.append(1/(1+np.exp(-(np.dot(genetic_train[i],train_model_coef)+train_model_intercept))))
y_pred=np.array(y_prob)
y_pred[y_pred>0.5]=1
y_pred[y_pred<=0.5]=0
y_pred = y_pred.astype('int64')
ocd_train = ocd_train.astype('int64')
print(accuracy_score(ocd_train,y_pred))
print(confusion_matrix(ocd_train,y_pred))

0.7071897174989958
[[2864 4374]
 [   0 7700]]


In [30]:
tn, fp, fn, tp = confusion_matrix(ocd_train,y_pred).ravel()
ck = (2*((tp * tn) - (fn * fp)))/(((tp + fp) * (fp + tn)) + ((tp + fn) * (fn+tp)))
print(ck)

0.30068935975424105
